In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

loader = WebBaseLoader(
    web_paths=("https://www.gutenberg.org/cache/epub/1399/pg1399-images.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("chapter", "pg-boilerplate pgheader", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
from langchain_chroma import Chroma

In [12]:
persist_directory='db'

In [ ]:
# vectorstore = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings(model='nomic-embed-text'),persist_directory=persist_directory)

In [13]:
vectorestore= Chroma(persist_directory=persist_directory,embedding_function=OllamaEmbeddings(model='nomic-embed-text'),)

In [14]:
query="what is the first name of oblonsky?"

In [16]:

retriever = vectorestore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("who is madame karenina")

'Based on the provided context, Madame Karenina appears to be a character in the story who is the subject of gossip and social controversy. She is mentioned as being seen with Alexey Vronsky and is involved in some kind of social incident with Madame Kartasova at what seems to be a theater or opera. However, the context does not provide enough information to definitively state who Madame Karenina is or her full role in the story.'

(40.357113, -3.757292)
(40.357113, -3.752292)
(40.357113, -3.7472920000000003)
(40.357113, -3.7422920000000004)
(40.357113, -3.7372920000000005)
(40.357113, -3.7322920000000006)
(40.357113, -3.7272920000000007)
(40.357113, -3.722292000000001)
(40.357113, -3.717292000000001)
(40.357113, -3.712292000000001)
Total coordinate points generated: 625


The distance is 423.67 meters


round 1
round 2
round 3
round 4
round 5
round 6
round 7
round 8
round 9
round 10
0


In [13]:
map_madrid = folium.Map(location=[40.4168, -3.7038], zoom_start=14)

for (index, row) in df.iterrows():
    folium.Marker(location = [row.loc["geometry.location.lat"], row.loc["geometry.location.lng"]]).add_to(map_madrid)

map_madrid

In [14]:
def trial_function():
    df=[]
    url=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=40.423889,-3.688381&radius=300&type=restaurant&type=bakery&type=bar&type=cafe&type=meal_delivery&type=meal_takeaway&key={os.getenv('GOOGLE_API_KEY')}"
    params={'pagetoken':''}
    while True:
        response= requests.get(url,params)
        if response.status_code != 200:
            print("There is a problem, please confirm the requests")
            break
        data= response.json()
        df.extend(data.get('results', []))
        next_page_token = data.get('next_page_token')
        if not next_page_token:
            break
        params['pagetoken'] = next_page_token
        
        time.sleep(2)
        
    return df

data= trial_function()
df = pd.json_normalize(data)
df.head()

,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,price_level,permanently_closed
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png,#909CE1,https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet,VP Jardín de Recoletos,"[{'height': 1333, 'html_attributions': ['<a href=""https://maps.google.com/maps/contrib/101318484028830464174"">VP Jardín de Recoletos</a>'], 'photo_reference': 'AelY_CsWNtYcJI_NnhV_zLMkI3fJe39_HM6-IRAahOKbc2v1TlbrX6Bb0ne016KQccu6XxvbNJ5Hh2_UhtLsAewnpN5Dq5O-aAHXAs1g5z9GhRmTu09cnVI9x3ZHCi7XE0Iht88FOeDcV6AAMlOrHq5RAzeJxm4BkJ3JyDVM_XnqQ_NK1kXa', 'width': 2000}]",ChIJ0U9LFZooQg0RgdlcHxZdkek,4.5,ChIJ0U9LFZooQg0RgdlcHxZdkek,GOOGLE,"[parking, restaurant, food, lodging, point_of_interest, establishment]",610.0,"Calle de Gil de Santivañes, 6, Madrid",40.422308,-3.689168,40.423685,-3.687793,40.420987,-3.690491,True,"C8C6+W8 Madrid, Spain",8CGRC8C6+W8,NaN,NaN
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png,#909CE1,https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet,ICON Wipton,"[{'height': 1080, 'html_attributions': ['<a href=""https://maps.google.com/maps/contrib/112263781301103430687"">ICON Wipton</a>'], 'photo_reference': 'AelY_CueL-6LFhMUT4Oo6ULv0gFE4f-byK2954oidKERyupCTLcPqa9Vjk-1CAjo7yhjxWLomweUyOx8HKr1lqjrV-07nY9e6OVoNBfe7JMl8rkmsVhaiM18N6zGS1ZR569khug-tHTUisdsgOg-nLb8ML7O7acKo2TAEL0oV0j26ZMp9tw7', 'width': 1620}]",ChIJXf7FfpcoQg0Ryp4nK6mgfw0,4.5,ChIJXf7FfpcoQg0Ryp4nK6mgfw0,GOOGLE,"[restaurant, food, lodging, point_of_interest, establishment]",577.0,"Calle de Jorge Juan, 17, Madrid",40.424027,-3.685417,40.425282,-3.683976,40.422584,-3.686674,NaN,"C8F7+JR Madrid, Spain",8CGRC8F7+JR,NaN,NaN
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,El Escondite de Villanueva,"[{'height': 1265, 'html_attributions': ['<a href=""https://maps.google.com/maps/contrib/108092814411675723884"">El Escondite de Villanueva</a>'], 'photo_reference': 'AelY_CsVWOqGrtqi2c3jLJSBqehZgY8ggJnpwXQksKjynoHVmdNvlsB0TmRYU0QPJ_RuLw8m7boRlIbi-JM3R9qGMS59uZ3kkZWIsSW4JeYdaMSagmruczttCr2J0K9JgdYzLd3-Ehjz8vYVQyF2MKt6mWpkrAHvwkMYMsk-KXwlAf-QQK6i', 'width': 2048}]",ChIJmzxvx5koQg0RRLZ2YkHb1g4,4.1,ChIJmzxvx5koQg0RRLZ2YkHb1g4,GOOGLE,"[restaurant, food, point_of_interest, establishment]",2349.0,"Calle de Villanueva, 26, Madrid",40.422366,-3.685980,40.423753,-3.684625,40.421055,-3.687323,False,"C8C7+WJ Madrid, Spain",8CGRC8C7+WJ,2.0,NaN
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,Restaurante La Giralda - Tu Rincón de Andalucía en Madrid -,"[{'height': 2160, 'html_attributions': ['<a href=""https://maps.google.com/maps/contrib/111699120058084729579"">Javier Valdés Valdés</a>'], 'photo_reference': 'AelY_CtUDfW8_4RInsoeXMSgT9oN0jO2iX7vvZ7m20BZE4V_aAK0fsFrNA22ySF80HD_e3oz4JS4mNqn8GEHxQHSeukfAuzP0IeN_RBepaLiDwoCdzwjNJQ6ZzXZrNw9nKUS20P6w0fhQt-hJw-mzwN-DyX1WJtQquvWr8MXHgFz16IbOANG', 'width': 3840}]",ChIJTz8O5JkoQg0R60u5D3X11EM,4.3,ChIJTz8O5JkoQg0R60u5D3X11EM,GOOGLE,"[restaurant, food, point_of_interest, establishment]",1461.0,"Calle de Claudio Coello, 24, Madrid",40.423265,-3.686598,40.424635,-3.685384,40.421938,-3.688082,False,"C8F7+89 Madrid, Spain",8CGRC8F7+89,2.0,NaN
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet,El Paraguas,"[{'height': 6000, 'html_attributions': ['<a hr